In [1]:
import keras 
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Flatten
from keras import optimizers
from keras.models import Sequential
from keras.layers import Input, Lambda, Dense, Flatten, BatchNormalization, Activation
from keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
# Training
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger
from keras.optimizers import SGD
from keras import backend as K

In [3]:
def AlexNet(input_shape,num_classes):
    
    X_input = Input(input_shape)
    
    X = Conv2D(96,(11,11),strides = 4,name="conv0")(X_input)
    X = BatchNormalization(axis = 3 , name = "bn0")(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max0')(X)
    
    X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
    X = BatchNormalization(axis = 3 ,name='bn1')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max1')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max2')(X)
    
    X = Flatten()(X)
    
    X = Dense(num_classes,activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs = X, name='AlexNet')
    return model

In [4]:
input_shape = (227, 227, 3)
num_classes = 7
AlexNet(input_shape, num_classes).summary()

Model: "AlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv0 (Conv2D)               (None, 55, 55, 96)        34944     
_________________________________________________________________
bn0 (BatchNormalization)     (None, 55, 55, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
max0 (MaxPooling2D)          (None, 27, 27, 96)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 27, 27, 256)       614656    
_________________________________________________________________
bn1 (BatchNormalization)     (None, 27, 27, 256)       1024

In [5]:
# Training
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger
from keras.optimizers import SGD
from keras import backend as K

In [6]:
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
trainset_dir = r"/content/drive/MyDrive/Dataset/Dataset_Shuffled/Train"
valset_dir = r"/content/drive/MyDrive/Dataset/Dataset_Shuffled/Validation"

In [8]:
num_classes = 7
learning_rate = 1e-3
batch_size = 8
input_shape = (227, 227, 3)
momentum = 0.9

In [9]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    trainset_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size)

val_generator = val_datagen.flow_from_directory(
    valset_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    shuffle=False)

Found 2677 images belonging to 7 classes.
Found 533 images belonging to 7 classes.


In [10]:
K.clear_session()

In [11]:
def precision(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    '''Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def accuracy(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    return K.mean(K.equal(y_true, K.round(y_pred)))

In [12]:
optim = SGD(lr=learning_rate, momentum=momentum)

In [13]:
model = AlexNet(input_shape, num_classes)

In [14]:
model.compile(loss='categorical_crossentropy',
                    optimizer=optim,
                    metrics=[precision,recall,'accuracy'])

In [15]:
csv_path = r"/content/drive/MyDrive/AlexNet/result_show/AlexNet.csv"
log_dir = r"/content/drive/MyDrive/AlexNet/result_show/AlexNet"
save_weights_path = r"/content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-{epoch:02d}-val_loss-{val_loss:.4f}-val_acc-{val_accuracy:.4f}.h5"

In [16]:
checkpoint = ModelCheckpoint(save_weights_path, monitor='val_accuracy', verbose=1, 
                             save_weights_only=True, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=20, verbose=1, min_lr=1e-4)

In [17]:
earlystop = EarlyStopping(monitor='val_accuracy', patience=25, verbose=1)

In [18]:
logging = TensorBoard(log_dir=log_dir)
csvlogger = CSVLogger(csv_path, append=True)
callbacks = [checkpoint, reduce_lr, logging, csvlogger]

In [19]:
num_epochs = 500

In [ ]:
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=num_epochs,
          verbose=1, 
          callbacks=callbacks, 
          validation_data=val_generator, 
          validation_steps=len(val_generator),
          workers = 1)

Epoch 1/500
335/335 [==============================] - 620s 2s/step - loss: 4.2372 - precision: 0.3057 - recall: 0.2384 - accuracy: 0.2845 - val_loss: 2.7279 - val_precision: 0.3666 - val_recall: 0.2519 - val_accuracy: 0.3358

Epoch 00001: val_accuracy improved from -inf to 0.33583, saving model to /content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-01-val_loss-2.7279-val_acc-0.3358.h5
Epoch 2/500
335/335 [==============================] - 42s 125ms/step - loss: 1.8777 - precision: 0.4749 - recall: 0.2794 - accuracy: 0.3883 - val_loss: 1.4803 - val_precision: 0.4648 - val_recall: 0.3090 - val_accuracy: 0.4484

Epoch 00002: val_accuracy improved from 0.33583 to 0.44841, saving model to /content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-02-val_loss-1.4803-val_acc-0.4484.h5
Epoch 3/500
335/335 [==============================] - 42s 125ms/step - loss: 1.6649 - precision: 0.5340 - recall: 0.3091 - accuracy: 0.4266 - val_loss: 1.6965 - val_precision: 0.4500 - val_recall: 0

In [7]:
#Testing
import numpy as np 
import tensorflow as tf 
import seaborn as sn
from matplotlib import pyplot as plt 
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator

In [8]:
import os

path = r"/content/drive/MyDrive/AlexNet/weights"
for file in os.listdir(path):
    filepath = os.path.join(path,file)
    input_shape = (227, 227, 3)
    num_classes = 7
    testset_dir = r"/content/drive/MyDrive/Dataset/Dataset_Shuffled/Testing"
    model = AlexNet(input_shape, num_classes)
    model.load_weights(filepath)
    # Prediction on test set
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    test_generator = test_datagen.flow_from_directory(
        testset_dir,
        target_size=(input_shape[0], input_shape[1]),
        batch_size=140 , # The number of test images
        class_mode='categorical')
    x_test, y_test = test_generator.__getitem__(0)
    y_pred = model.predict(x_test)
    acc = np.count_nonzero(np.equal(np.argmax(y_pred, axis=1), np.argmax(y_test, axis=1)))/x_test.shape[0]
    print(filepath)
    print(round(acc,2))# Test set accuracy

Found 140 images belonging to 7 classes.
/content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-01-val_loss-2.7279-val_acc-0.3358.h5
0.34
Found 140 images belonging to 7 classes.
/content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-02-val_loss-1.4803-val_acc-0.4484.h5
0.46
Found 140 images belonging to 7 classes.
/content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-05-val_loss-1.5329-val_acc-0.4522.h5
0.47
Found 140 images belonging to 7 classes.
/content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-07-val_loss-1.4300-val_acc-0.4784.h5
0.51
Found 140 images belonging to 7 classes.
/content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-11-val_loss-1.2271-val_acc-0.5722.h5
0.54
Found 140 images belonging to 7 classes.
/content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-14-val_loss-1.0581-val_acc-0.6135.h5
0.66
Found 140 images belonging to 7 classes.
/content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-17-val_loss-0.9417-val_acc-0.6454.

In [ ]:
input_shape = (227, 227, 3)
num_classes = 7
testset_dir = r"/content/drive/MyDrive/Dataset/Testing"
weight_path =  r"/content/drive/MyDrive/AlexNet/weights/trash-model-weight-ep-150-val_loss-0.5162-val_acc-0.9272.h5"
model = AlexNet(input_shape, num_classes)
model.load_weights(weight_path)

In [ ]:
# Prediction on test set
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    testset_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=134 , # The number of test images
    class_mode='categorical')

In [ ]:
x_test, y_test = test_generator.__getitem__(0)

In [ ]:
test_true = np.argmax(y_test, axis=1)
test_pred = np.argmax(model.predict(x_test), axis=1)
print("CNN Model Accuracy on test set: {:.4f}".format(accuracy_score(test_true, test_pred)))
print(classification_report(test_true, test_pred))

In [ ]:
y_pred = model.predict(x_test)
acc = np.count_nonzero(np.equal(np.argmax(y_pred, axis=1), np.argmax(y_test, axis=1)))/x_test.shape[0]
print(round(acc,2))# Test set accuracy